In [ ]:
from google.colab import files
uploaded = files.upload()  # Choose files from your system


Saving updated_iemocap_metadata.csv to updated_iemocap_metadata.csv


In [ ]:
!ls

sample_data  updated_iemocap_metadata.csv


In [ ]:
import pandas as pd

# Step 1: Load the CSV
csv_path = '/content/updated_iemocap_metadata.csv'
df = pd.read_csv(csv_path)

# Step 2: Update paths (from Kaggle path to Colab path)
df['filepath'] = df['filepath'].str.replace(
    '/kaggle/input/iemocap/iemocap_audio/', '/content/iemocap/', regex=False
)

# Step 3: Save the updated CSV
updated_csv_path = '/content/sample_data/updated_iemocap_metadata_colab.csv'
df.to_csv(updated_csv_path, index=False)

print(f"✅ Updated CSV saved to: {updated_csv_path}")


✅ Updated CSV saved to: /content/sample_data/updated_iemocap_metadata_colab.csv


In [ ]:
print(df['filepath'].head())

0    /content/iemocap/Ses01F_impro01_F000.wav
1    /content/iemocap/Ses01F_impro01_F001.wav
2    /content/iemocap/Ses01F_impro01_F002.wav
3    /content/iemocap/Ses01F_impro01_F003.wav
4    /content/iemocap/Ses01F_impro01_F004.wav
Name: filepath, dtype: object


In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive')


Drive not mounted, so nothing to flush and unmount.
Mounted at /content/drive


In [ ]:
import zipfile
import os

# Set the path
zip_path = '/content/drive/MyDrive/iemocap_audio.zip'
extract_to = '/content/iemocap/'

# Create target folder
os.makedirs(extract_to, exist_ok=True)

# Extract the zip
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"✅ Unzipped to {extract_to}")


✅ Unzipped to /content/iemocap/


In [ ]:
import pandas as pd
import librosa
from IPython.display import Audio

# Load CSV
csv_path = '/content/sample_data/updated_iemocap_metadata_colab.csv'
df = pd.read_csv(csv_path)

# Get the first audio file path
audio_path = df['filepath'].iloc[0]

# Load audio
waveform, sr = librosa.load(audio_path, sr=None)

# Play audio
Audio(waveform, rate=sr)


In [ ]:
# Step 2: Install dependencies
!pip install numpy==1.26.4
!pip install pandas==2.2.2
!pip install torch==2.0.0 torchvision==0.15.1
!pip install transformers==4.31.0
!pip install torchaudio==2.0.1
!pip install tqdm==4.66.2
!pip install nlpaug==1.1.11

In [ ]:
import numpy
import pandas
import torch
import transformers
import torchaudio
import tqdm
import nlpaug
print(f"NumPy: {numpy.__version__}")  # Should be 1.26.4
print(f"Pandas: {pandas.__version__}")  # Should be 2.2.2
print(f"Torch: {torch.__version__}")  # Should be 2.0.0
print(f"Transformers: {transformers.__version__}")  # Should be 4.31.0
print(f"Torchaudio: {torchaudio.__version__}")  # Should be 2.0.1
print(f"Tqdm: {tqdm.__version__}")  # Should be 4.66.2
print(f"Nlpaug: {nlpaug.__version__}")  # Should be 1.1.11
print(f"CUDA available: {torch.cuda.is_available()}")  # Should be True

NumPy: 1.26.4
Pandas: 2.2.2
Torch: 2.0.0+cu117
Transformers: 4.31.0
Torchaudio: 2.0.1+cu117
Tqdm: 4.66.2
Nlpaug: 1.1.11
CUDA available: False


In [ ]:
# Step 2: Initialize NLTK resources
import nltk
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from transformers import HubertModel, Wav2Vec2FeatureExtractor, BertModel, BertTokenizer
import torchaudio
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import nlpaug.augmenter.word as naw
import random

# Dataset with fixed augmentation
class MultimodalIEMOCAPDataset(Dataset):
    def __init__(self, csv_file, audio_feature_extractor, text_tokenizer, augment=True, max_audio_samples=128000):
        self.df = pd.read_csv(csv_file)
        self.audio_feature_extractor = audio_feature_extractor
        self.text_tokenizer = text_tokenizer
        self.augment = augment
        self.text_augmenter = naw.SynonymAug(aug_p=0.3) if augment else None
        self.max_audio_samples = max_audio_samples

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]['filepath']
        transcript = self.df.iloc[idx]['transcription']
        vad_label = self.df.iloc[idx]['EmoVal'].astype(np.float32)

        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio file missing: {audio_path}")

        try:
            audio, sr = torchaudio.load(audio_path)
            if audio.abs().mean() < 1e-5:
                raise ValueError(f"Silent audio: {audio_path}")
        except Exception as e:
            raise RuntimeError(f"Error loading audio {audio_path}: {str(e)}")

        if sr != 16000:
            audio = torchaudio.transforms.Resample(sr, 16000)(audio)

        audio = audio.squeeze(0)  # Remove channel dim if mono
        if audio.dim() > 1:
            audio = audio[0]  # Take first channel if stereo

        # Pad or truncate to max_audio_samples
        if audio.size(0) > self.max_audio_samples:
            audio = audio[:self.max_audio_samples]
        elif audio.size(0) < self.max_audio_samples:
            audio = torch.nn.functional.pad(audio, (0, self.max_audio_samples - audio.size(0)))

        if self.augment and random.random() < 0.5:
            # Add noise
            noise = torch.randn_like(audio) * 0.005
            audio = audio + noise

            # Speed augmentation
            try:
                speed_factor = random.uniform(0.9, 1.1)
                effect = torch.tensor(audio).unsqueeze(0)  # [1, samples]
                augmented_audio, new_sr = torchaudio.sox_effects.apply_effects_tensor(
                    effect,
                    sample_rate=16000,
                    effects=[["speed", str(speed_factor)], ["rate", "16000"]]
                )
                audio = augmented_audio.squeeze(0)
                # Re-pad or truncate to max_audio_samples
                if audio.size(0) > self.max_audio_samples:
                    audio = audio[:self.max_audio_samples]
                elif audio.size(0) < self.max_audio_samples:
                    audio = torch.nn.functional.pad(audio, (0, self.max_audio_samples - audio.size(0)))
            except Exception as e:
                print(f"Speed augmentation failed for {audio_path}: {e}")
                # Keep original audio

        audio = audio.numpy()

        if self.augment and self.text_augmenter and random.random() < 0.3:
            try:
                transcript = self.text_augmenter.augment(transcript)[0]
            except Exception as e:
                print(f"Text augmentation failed for {audio_path}: {e}")
                # Keep original transcript

        audio_inputs = self.audio_feature_extractor(
            audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=False,
            truncation=False
        )

        text_inputs = self.text_tokenizer(
            transcript,
            padding=False,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        return {
            'audio_values': audio_inputs['input_values'].squeeze(0),
            'input_ids': text_inputs['input_ids'].squeeze(0),
            'attention_mask': text_inputs['attention_mask'].squeeze(0)
        }, torch.tensor(vad_label)

# Dynamic collation
def dynamic_collate_fn(batch):
    inputs, labels = zip(*batch)
    audio_values = [item['audio_values'] for item in inputs]
    input_ids = [item['input_ids'] for item in inputs]
    attention_masks = [item['attention_mask'] for item in inputs]

    audio_values_padded = pad_sequence(audio_values, batch_first=True, padding_value=0.0)
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_masks_padded = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    labels_stacked = torch.stack(labels)

    return {
        'audio_values': audio_values_padded,
        'input_ids': input_ids_padded,
        'attention_mask': attention_masks_padded
    }, labels_stacked

# Label smoothing loss
class SmoothMSELoss(nn.Module):
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing
        self.mse = nn.MSELoss()

    def forward(self, pred, target):
        smooth_target = target * (1 - self.smoothing) + 3.0 * self.smoothing
        return self.mse(pred, smooth_target)

# Transformer model
class ValenceRegressor(nn.Module):
    def __init__(self, audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5):
        super().__init__()

        self.audio_transformer = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=audio_dim,
                nhead=num_heads,
                dim_feedforward=hidden_dim * 4,
                dropout=dropout,
                batch_first=True
            ) for _ in range(num_layers)
        ])

        self.audio_layer_norm = nn.LayerNorm(audio_dim)

        self.audio_attention_pool = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, 1)
        )

        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        for param in list(self.text_encoder.parameters())[-2:]:
            param.requires_grad = True

        self.audio_projection = nn.Linear(audio_dim, hidden_dim)
        self.text_projection = nn.Linear(text_dim, hidden_dim)

        self.audio_to_text_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads//2,
            dropout=dropout,
            batch_first=True
        )

        self.text_to_audio_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads//2,
            dropout=dropout,
            batch_first=True
        )

        self.audio_gate = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.Sigmoid()
        )

        self.text_gate = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.Sigmoid()
        )

        self.fusion_layer = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.LayerNorm(hidden_dim*2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim*2, hidden_dim)
        )

        self.shared_fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )

        self.output_branch = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.LayerNorm(hidden_dim//2),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dim//2, 1)
        )

    def audio_attention_pooling(self, x, audio_mask=None):
        weights = self.audio_attention_pool(x)
        if audio_mask is not None:
            weights = weights.masked_fill(~audio_mask.bool().unsqueeze(-1), float('-inf'))
        weights = torch.softmax(weights, dim=1)
        output = torch.bmm(weights.transpose(1, 2), x)
        return output.squeeze(1)

    def forward(self, audio_features, input_ids, attention_mask):
        audio_mask = (audio_features.abs().sum(dim=-1) > 1e-6)

        audio_repr = audio_features
        for layer in self.audio_transformer:
            audio_key_padding_mask = (~audio_mask).float()
            audio_repr = layer(audio_repr, src_key_padding_mask=audio_key_padding_mask)

        audio_repr = self.audio_layer_norm(audio_repr)

        text_outputs = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        text_repr = text_outputs.last_hidden_state

        audio_proj = self.audio_projection(audio_repr)
        text_proj = self.text_projection(text_repr)

        audio_attended_text, _ = self.audio_to_text_attention(
            query=audio_proj,
            key=text_proj,
            value=text_proj,
            key_padding_mask=(1 - attention_mask).bool()
        )

        text_attended_audio, _ = self.text_to_audio_attention(
            query=text_proj,
            key=audio_proj,
            value=audio_proj,
            key_padding_mask=(~audio_mask).bool()
        )

        audio_concat = torch.cat([audio_proj, audio_attended_text], dim=-1)
        text_concat = torch.cat([text_proj, text_attended_audio], dim=-1)

        audio_gate_value = self.audio_gate(audio_concat)
        text_gate_value = self.text_gate(text_concat)

        gated_audio = audio_proj * audio_gate_value
        gated_text = text_proj * text_gate_value

        pooled_audio = self.audio_attention_pooling(gated_audio, audio_mask)
        text_sum = torch.sum(gated_text * attention_mask.unsqueeze(-1), dim=1)
        text_count = torch.sum(attention_mask, dim=1, keepdim=True).clamp(min=1)
        pooled_text = text_sum / text_count

        fused = torch.cat([pooled_audio, pooled_text], dim=1)
        joint_repr = self.fusion_layer(fused)

        shared = self.shared_fc(joint_repr)

        output = self.output_branch(shared)
        scaled_output = 1.0 + 4.0 * torch.sigmoid(output)

        return scaled_output

# Training function
def train_valence_model(model, train_loader, val_loader, audio_model,
                        num_epochs=15, lr=5e-5, max_norm=0.5):
    device = torch.device("cuda")
    model = model.to(device)
    audio_model = audio_model.to(device)

    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-2)
    criterion = SmoothMSELoss(smoothing=0.1)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=3, T_mult=2, eta_min=1e-6
    )

    best_val_loss = float('inf')
    patience_counter = 0
    max_patience = 7

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        num_train_batches = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train Valence]")

        for batch_inputs, labels in train_pbar:
            try:
                audio_values = batch_inputs['audio_values'].to(device)
                input_ids = batch_inputs['input_ids'].to(device)
                attention_mask = batch_inputs['attention_mask'].to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.no_grad():
                    hubert_out = audio_model(audio_values).last_hidden_state
                    hubert_out = hubert_out.detach()

                outputs = model(hubert_out, input_ids, attention_mask)
                loss = criterion(outputs.squeeze(), labels)

                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
                optimizer.step()

                train_loss += loss.item()
                num_train_batches += 1

                train_pbar.set_postfix({'loss': f"{loss.item():.4f}"})

            except Exception as e:
                print(f"Error in training batch: {e}")
                continue

        scheduler.step()

        if num_train_batches > 0:
            train_loss /= num_train_batches

        model.eval()
        val_loss = 0.0
        num_val_batches = 0

        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val Valence]")

        with torch.no_grad():
            for batch_inputs, labels in val_pbar:
                try:
                    audio_values = batch_inputs['audio_values'].to(device)
                    input_ids = batch_inputs['input_ids'].to(device)
                    attention_mask = batch_inputs['attention_mask'].to(device)
                    labels = labels.to(device)

                    hubert_out = audio_model(audio_values).last_hidden_state

                    outputs = model(hubert_out, input_ids, attention_mask)
                    loss = criterion(outputs.squeeze(), labels)

                    val_loss += loss.item()
                    num_val_batches += 1

                    val_pbar.set_postfix({'loss': f"{loss.item():.4f}"})

                except Exception as e:
                    print(f"Error in validation batch: {e}")
                    continue

        if num_val_batches > 0:
            val_loss /= num_val_batches

        print(f"Epoch {epoch+1}/{num_epochs} [Valence] Results:")
        print(f"  Train Loss: {train_loss:.4f}")
        print(f"  Val Loss: {val_loss:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), '/content/best_valence_regressor.pth')
            print(f"  Saved best model (val_loss: {val_loss:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= max_patience:
                print(f"Early stopping after {epoch+1} epochs without improvement")
                break

    return best_val_loss

# Evaluation function
def evaluate_valence_model(model, test_loader, audio_model, device):
    model.eval()
    criterion = SmoothMSELoss(smoothing=0.1)
    test_loss = 0.0
    num_batches = 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch_inputs, labels in test_loader:
            try:
                audio_values = batch_inputs['audio_values'].to(device)
                input_ids = batch_inputs['input_ids'].to(device)
                attention_mask = batch_inputs['attention_mask'].to(device)
                labels = labels.to(device)

                hubert_out = audio_model(audio_values).last_hidden_state

                outputs = model(hubert_out, input_ids, attention_mask)
                loss = criterion(outputs.squeeze(), labels)

                test_loss += loss.item()
                num_batches += 1

                all_preds.append(outputs.squeeze().cpu().numpy())
                all_labels.append(labels.cpu().numpy())

            except Exception as e:
                print(f"Error in test batch: {e}")
                continue

    if num_batches > 0:
        test_loss /= num_batches
        all_preds = np.concatenate(all_preds)
        all_labels = np.concatenate(all_labels)
        from scipy.stats import pearsonr
        pcc, _ = pearsonr(all_preds, all_labels)
    else:
        test_loss = float('inf')
        pcc = 0.0

    return test_loss, pcc

# Main execution
def main():
    device = torch.device("cuda")

    # Configuration
    csv_file = "/content/sample_data/updated_iemocap_metadata_colab.csv"
    audio_feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
    text_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    hubert_model = HubertModel.from_pretrained("facebook/hubert-base-ls960")
    for param in hubert_model.parameters():
        param.requires_grad = False
    hubert_model.eval()
    hubert_model = hubert_model.to(device)

    # Dataset and loaders
    train_dataset = MultimodalIEMOCAPDataset(
        csv_file=csv_file,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=True,
        max_audio_samples=128000
    )
    val_dataset = MultimodalIEMOCAPDataset(
        csv_file=csv_file,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=False,
        max_audio_samples=128000
    )
    test_dataset = MultimodalIEMOCAPDataset(
        csv_file=csv_file,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=False,
        max_audio_samples=128000
    )

    train_size = int(0.8 * len(train_dataset))
    val_size = int(0.1 * len(train_dataset))
    test_size = len(train_dataset) - train_size - val_size
    train_dataset, _, _ = random_split(
        train_dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    _, val_dataset, _ = random_split(
        val_dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    _, _, test_dataset = random_split(
        test_dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        collate_fn=dynamic_collate_fn
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=dynamic_collate_fn
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=dynamic_collate_fn
    )

    # Model
    model = ValenceRegressor(
        audio_dim=768,
        text_dim=768,
        hidden_dim=192,
        num_heads=6,
        num_layers=2,
        dropout=0.5
    ).to(device)

    # Training
    print("Training Valence model...")
    best_val_loss = train_valence_model(
        model,
        train_loader,
        val_loader,
        hubert_model,
        num_epochs=15,
        lr=5e-5,
        max_norm=0.5
    )

    # Evaluation
    model.load_state_dict(torch.load('/content/best_valence_regressor.pth'))
    test_loss, test_pcc = evaluate_valence_model(model, test_loader, hubert_model, device)

    print("\nTest Results:")
    print(f"  Valence MSE: {test_loss:.4f}")
    print(f"  Valence PCC: {test_pcc:.4f}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training Valence model...


Epoch 1/15 [Train Valence]:   0%|          | 0/1004 [00:00<?, ?it/s]<ipython-input-10-dde1b6da82b4>:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  effect = torch.tensor(audio).unsqueeze(0)  # [1, samples]
Epoch 1/15 [Val Valence]:   0%|          | 0/126 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:544: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(
Epoch 1/15 [Val Valence]: 100%|██████████| 126/126 [00:45<00:00,  2.76it/s, loss=0.4118]


Epoch 1/15 [Valence] Results:
  Train Loss: 0.6754
  Val Loss: 0.4813
  Saved best model (val_loss: 0.4813)


Epoch 2/15 [Val Valence]: 100%|██████████| 126/126 [00:44<00:00,  2.82it/s, loss=0.1743]


...

Epoch 13/15 [Val Valence]: 100%|██████████| 126/126 [00:44<00:00,  2.84it/s, loss=0.0209]


Epoch 13/15 [Valence] Results:
  Train Loss: 0.2726
  Val Loss: 0.3717


Epoch 14/15 [Val Valence]: 100%|██████████| 126/126 [00:44<00:00,  2.84it/s, loss=0.0028]


Epoch 14/15 [Valence] Results:
  Train Loss: 0.2543
  Val Loss: 0.3121
  Saved best model (val_loss: 0.3121)


Epoch 15/15 [Train Valence]:   1%|          | 8/1004 [00:04<07:50,  2.11it/s, loss=0.2920]

In [ ]:
!ls

sample_data


In [ ]:
import torch

model_path = '/content/drive/MyDrive/best_valence_regressor_8.pth'
model = torch.load(model_path, map_location='cpu')


In [ ]:
import torch
import torchaudio
from transformers import HubertModel, Wav2Vec2FeatureExtractor, BertModel, BertTokenizer
import pandas as pd
import numpy as np
import os

def predict_valence_from_csv(csv_file, row_idx, model_path, device='cpu'):
    # Load the CSV file
    df = pd.read_csv(csv_file)
    if row_idx < 0 or row_idx >= len(df):
        raise ValueError(f"Row index {row_idx} is out of range [0, {len(df)-1}]")

    # Extract audio path and transcription from the specified row
    audio_path = df.iloc[row_idx]['filepath']
    transcription = df.iloc[row_idx]['transcription']
    ground_truth_valence = df.iloc[row_idx]['EmoVal'].astype(np.float32)

    # Load the trained model with map_location to CPU
    device = torch.device(device)  # Force CPU since CUDA is unavailable
    model = ValenceRegressor(audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5)
    model.load_state_dict(torch.load(model_path, map_location=device))  # Map to CPU
    model = model.to(device)
    model.eval()

    # Load feature extractors and tokenizers
    audio_feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
    text_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    hubert_model = HubertModel.from_pretrained("facebook/hubert-base-ls960")
    hubert_model.eval()
    hubert_model = hubert_model.to(device)
    for param in hubert_model.parameters():
        param.requires_grad = False

    # Load and preprocess audio
    if not os.path.exists(audio_path):
        raise FileNotFoundError(f"Audio file missing: {audio_path}")

    try:
        audio, sr = torchaudio.load(audio_path)
        if audio.abs().mean() < 1e-5:
            raise ValueError(f"Silent audio: {audio_path}")
    except Exception as e:
        raise RuntimeError(f"Error loading audio {audio_path}: {str(e)}")

    if sr != 16000:
        audio = torchaudio.transforms.Resample(sr, 16000)(audio)

    audio = audio.squeeze(0)  # Remove channel dim if mono
    if audio.dim() > 1:
        audio = audio[0]  # Take first channel if stereo

    max_audio_samples = 128000
    if audio.size(0) > max_audio_samples:
        audio = audio[:max_audio_samples]
    elif audio.size(0) < max_audio_samples:
        audio = torch.nn.functional.pad(audio, (0, max_audio_samples - audio.size(0)))

    audio = audio.numpy()
    audio_inputs = audio_feature_extractor(
        audio,
        sampling_rate=16000,
        return_tensors="pt",
        padding=False,
        truncation=False
    )
    audio_values = audio_inputs['input_values'].to(device)

    # Process transcription
    text_inputs = text_tokenizer(
        transcription,
        padding=False,
        truncation=True,
        max_length=512,
        return_tensors="pt"
    )
    input_ids = text_inputs['input_ids'].to(device)
    attention_mask = text_inputs['attention_mask'].to(device)

    # Get features from Hubert model
    with torch.no_grad():
        hubert_out = hubert_model(audio_values).last_hidden_state

    # Predict valence
    with torch.no_grad():
        output = model(hubert_out, input_ids, attention_mask)
        predicted_valence = output.squeeze().item()  # Scale is 1.0 to 5.0

    return predicted_valence, ground_truth_valence

# Transformer model definition (same as in training code)
class ValenceRegressor(nn.Module):
    def __init__(self, audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5):
        super().__init__()

        self.audio_transformer = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=audio_dim,
                nhead=num_heads,
                dim_feedforward=hidden_dim * 4,
                dropout=dropout,
                batch_first=True
            ) for _ in range(num_layers)
        ])

        self.audio_layer_norm = nn.LayerNorm(audio_dim)

        self.audio_attention_pool = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, 1)
        )

        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        for param in list(self.text_encoder.parameters())[-2:]:
            param.requires_grad = True

        self.audio_projection = nn.Linear(audio_dim, hidden_dim)
        self.text_projection = nn.Linear(text_dim, hidden_dim)

        self.audio_to_text_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads//2,
            dropout=dropout,
            batch_first=True
        )

        self.text_to_audio_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads//2,
            dropout=dropout,
            batch_first=True
        )

        self.audio_gate = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.Sigmoid()
        )

        self.text_gate = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.Sigmoid()
        )

        self.fusion_layer = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.LayerNorm(hidden_dim*2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim*2, hidden_dim)
        )

        self.shared_fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )

        self.output_branch = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.LayerNorm(hidden_dim//2),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dim//2, 1)
        )

    def audio_attention_pooling(self, x, audio_mask=None):
        weights = self.audio_attention_pool(x)
        if audio_mask is not None:
            weights = weights.masked_fill(~audio_mask.bool().unsqueeze(-1), float('-inf'))
        weights = torch.softmax(weights, dim=1)
        output = torch.bmm(weights.transpose(1, 2), x)
        return output.squeeze(1)

    def forward(self, audio_features, input_ids, attention_mask):
        audio_mask = (audio_features.abs().sum(dim=-1) > 1e-6)

        audio_repr = audio_features
        for layer in self.audio_transformer:
            audio_key_padding_mask = (~audio_mask).float()
            audio_repr = layer(audio_repr, src_key_padding_mask=audio_key_padding_mask)

        audio_repr = self.audio_layer_norm(audio_repr)

        text_outputs = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        text_repr = text_outputs.last_hidden_state

        audio_proj = self.audio_projection(audio_repr)
        text_proj = self.text_projection(text_repr)

        audio_attended_text, _ = self.audio_to_text_attention(
            query=audio_proj,
            key=text_proj,
            value=text_proj,
            key_padding_mask=(1 - attention_mask).bool()
        )

        text_attended_audio, _ = self.text_to_audio_attention(
            query=text_proj,
            key=audio_proj,
            value=audio_proj,
            key_padding_mask=(~audio_mask).bool()
        )

        audio_concat = torch.cat([audio_proj, audio_attended_text], dim=-1)
        text_concat = torch.cat([text_proj, text_attended_audio], dim=-1)

        audio_gate_value = self.audio_gate(audio_concat)
        text_gate_value = self.text_gate(text_concat)

        gated_audio = audio_proj * audio_gate_value
        gated_text = text_proj * text_gate_value

        pooled_audio = self.audio_attention_pooling(gated_audio, audio_mask)
        text_sum = torch.sum(gated_text * attention_mask.unsqueeze(-1), dim=1)
        text_count = torch.sum(attention_mask, dim=1, keepdim=True).clamp(min=1)
        pooled_text = text_sum / text_count

        fused = torch.cat([pooled_audio, pooled_text], dim=1)
        joint_repr = self.fusion_layer(fused)

        shared = self.shared_fc(joint_repr)

        output = self.output_branch(shared)
        scaled_output = 1.0 + 4.0 * torch.sigmoid(output)

        return scaled_output

# Example usage
if __name__ == "__main__":
    # Edit these paths and row index
    csv_file = "/content/sample_data/updated_iemocap_metadata_colab.csv"
    row_idx = 10021  # Change to desired row number
    model_path = "/content/drive/MyDrive/best_valence_regressor_8.pth"

    predicted_valence, ground_truth_valence = predict_valence_from_csv(csv_file, row_idx, model_path)
    print(f"Predicted Valence Score: {predicted_valence:.4f}")
    print(f"Ground Truth Valence: {ground_truth_valence:.4f}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:544: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._transformer_encoder_layer_fwd(


Predicted Valence Score: 1.7735
Ground Truth Valence: 1.5000


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from transformers import HubertModel, Wav2Vec2FeatureExtractor, BertModel, BertTokenizer, get_linear_schedule_with_warmup
import torchaudio
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import nlpaug.augmenter.word as naw
import random

# Dataset with fixed tensor copy
class MultimodalIEMOCAPDataset(Dataset):
    def __init__(self, csv_file, audio_feature_extractor, text_tokenizer, augment=True, max_audio_samples=128000):
        self.df = pd.read_csv(csv_file)
        self.audio_feature_extractor = audio_feature_extractor
        self.text_tokenizer = text_tokenizer
        self.augment = augment
        self.text_augmenter = naw.SynonymAug(aug_p=0.3) if augment else None
        self.max_audio_samples = max_audio_samples

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        audio_path = self.df.iloc[idx]['filepath']
        transcript = self.df.iloc[idx]['transcription']

        # Get both arousal and dominance labels
        arousal_label = self.df.iloc[idx]['EmoAct'].astype(np.float32)
        dominance_label = self.df.iloc[idx]['EmoDom'].astype(np.float32)

        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio file missing: {audio_path}")

        try:
            audio, sr = torchaudio.load(audio_path)
            if audio.abs().mean() < 1e-5:
                raise ValueError(f"Silent audio: {audio_path}")
        except Exception as e:
            raise RuntimeError(f"Error loading audio {audio_path}: {str(e)}")

        if sr != 16000:
            audio = torchaudio.transforms.Resample(sr, 16000)(audio)

        audio = audio.squeeze(0)  # Remove channel dim if mono
        if audio.dim() > 1:
            audio = audio[0]  # Take first channel if stereo

        # Pad or truncate to max_audio_samples
        if audio.size(0) > self.max_audio_samples:
            audio = audio[:self.max_audio_samples]
        elif audio.size(0) < self.max_audio_samples:
            audio = torch.nn.functional.pad(audio, (0, self.max_audio_samples - audio.size(0)))

        if self.augment and random.random() < 0.5:
            # Add noise
            noise = torch.randn_like(audio) * 0.005
            audio = audio + noise

            # Speed augmentation
            try:
                speed_factor = random.uniform(0.9, 1.1)
                effect = audio.clone().detach().unsqueeze(0)  # [1, samples]
                augmented_audio, new_sr = torchaudio.sox_effects.apply_effects_tensor(
                    effect,
                    sample_rate=16000,
                    effects=[["speed", str(speed_factor)], ["rate", "16000"]]
                )
                audio = augmented_audio.squeeze(0)
                # Re-pad or truncate to max_audio_samples
                if audio.size(0) > self.max_audio_samples:
                    audio = audio[:self.max_audio_samples]
                elif audio.size(0) < self.max_audio_samples:
                    audio = torch.nn.functional.pad(audio, (0, self.max_audio_samples - audio.size(0)))
            except Exception as e:
                print(f"Speed augmentation failed for {audio_path}: {e}")
                # Keep original audio

        audio = audio.numpy()

        if self.augment and self.text_augmenter and random.random() < 0.3:
            try:
                transcript = self.text_augmenter.augment(transcript)[0]
            except Exception as e:
                print(f"Text augmentation failed for {audio_path}: {e}")
                # Keep original transcript

        audio_inputs = self.audio_feature_extractor(
            audio,
            sampling_rate=16000,
            return_tensors="pt",
            padding=False,
            truncation=False
        )

        text_inputs = self.text_tokenizer(
            transcript,
            padding=False,
            truncation=True,
            max_length=512,
            return_tensors="pt"
        )

        return {
            'audio_values': audio_inputs['input_values'].squeeze(0),
            'input_ids': text_inputs['input_ids'].squeeze(0),
            'attention_mask': text_inputs['attention_mask'].squeeze(0)
        }, {
            'arousal': torch.tensor(arousal_label),
            'dominance': torch.tensor(dominance_label)
        }

# Dynamic collation
def dynamic_collate_fn(batch):
    inputs, labels = zip(*batch)
    audio_values = [item['audio_values'] for item in inputs]
    input_ids = [item['input_ids'] for item in inputs]
    attention_masks = [item['attention_mask'] for item in inputs]

    audio_values_padded = pad_sequence(audio_values, batch_first=True, padding_value=0.0)
    input_ids_padded = pad_sequence(input_ids, batch_first=True, padding_value=0)
    attention_masks_padded = pad_sequence(attention_masks, batch_first=True, padding_value=0)

    arousal_labels = torch.stack([item['arousal'] for item in labels])
    dominance_labels = torch.stack([item['dominance'] for item in labels])

    return {
        'audio_values': audio_values_padded,
        'input_ids': input_ids_padded,
        'attention_mask': attention_masks_padded
    }, {
        'arousal': arousal_labels,
        'dominance': dominance_labels
    }

# MultimodalArousalDominanceModel with fixed audio_attention_pool
class MultimodalArousalDominanceModel(nn.Module):
    def __init__(self, audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5):
        super().__init__()

        self.audio_transformer = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=audio_dim,
                nhead=num_heads,
                dim_feedforward=hidden_dim * 4,
                dropout=dropout,
                batch_first=True
            ) for _ in range(num_layers)
        ])

        self.audio_layer_norm = nn.LayerNorm(audio_dim)

        # Fixed input dimension to hidden_dim
        self.audio_attention_pool = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim * 2),  # 192 -> 384
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim * 2, 1)  # 384 -> 1
        )

        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        for param in list(self.text_encoder.parameters())[-2:]:
            param.requires_grad = True

        self.audio_projection = nn.Linear(audio_dim, hidden_dim)
        self.text_projection = nn.Linear(text_dim, hidden_dim)

        self.audio_to_text_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads//2,
            dropout=dropout,
            batch_first=True
        )

        self.text_to_audio_attention = nn.MultiheadAttention(
            embed_dim=hidden_dim,
            num_heads=num_heads//2,
            dropout=dropout,
            batch_first=True
        )

        self.audio_gate = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.Sigmoid()
        )

        self.text_gate = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim),
            nn.Sigmoid()
        )

        # Task-Specific Fusion Layers
        self.fusion_layer_arousal = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.LayerNorm(hidden_dim*2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim*2, hidden_dim)
        )

        self.fusion_layer_dominance = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2),
            nn.LayerNorm(hidden_dim*2),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim*2, hidden_dim)
        )

        # Task-Specific FC Layers
        self.shared_fc_arousal = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )

        self.shared_fc_dominance = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout)
        )

        # Task-Specific Output Branches
        self.output_branch_arousal = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.LayerNorm(hidden_dim//2),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dim//2, 1)
        )

        self.output_branch_dominance = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2),
            nn.LayerNorm(hidden_dim//2),
            nn.GELU(),
            nn.Dropout(dropout * 0.5),
            nn.Linear(hidden_dim//2, 1)
        )

    def audio_attention_pooling(self, x, audio_mask=None):
        weights = self.audio_attention_pool(x)
        if audio_mask is not None:
            weights = weights.masked_fill(~audio_mask.bool().unsqueeze(-1), float('-inf'))
        weights = torch.softmax(weights, dim=1)
        output = torch.bmm(weights.transpose(1, 2), x)
        return output.squeeze(1)

    def forward(self, audio_features, input_ids, attention_mask):
        audio_mask = (audio_features.abs().sum(dim=-1) > 1e-6)

        audio_repr = audio_features
        for layer in self.audio_transformer:
            audio_key_padding_mask = (~audio_mask).float()
            audio_repr = layer(audio_repr, src_key_padding_mask=audio_key_padding_mask)

        audio_repr = self.audio_layer_norm(audio_repr)

        text_outputs = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        text_repr = text_outputs.last_hidden_state

        audio_proj = self.audio_projection(audio_repr)
        text_proj = self.text_projection(text_repr)

        audio_attended_text, _ = self.audio_to_text_attention(
            query=audio_proj,
            key=text_proj,
            value=text_proj,
            key_padding_mask=(1 - attention_mask).bool()
        )

        text_attended_audio, _ = self.text_to_audio_attention(
            query=text_proj,
            key=audio_proj,
            value=audio_proj,
            key_padding_mask=(~audio_mask).bool()
        )

        audio_concat = torch.cat([audio_proj, audio_attended_text], dim=-1)
        text_concat = torch.cat([text_proj, text_attended_audio], dim=-1)

        audio_gate_value = self.audio_gate(audio_concat)
        text_gate_value = self.text_gate(text_concat)

        gated_audio = audio_proj * audio_gate_value
        gated_text = text_proj * text_gate_value

        pooled_audio = self.audio_attention_pooling(gated_audio, audio_mask)
        text_sum = torch.sum(gated_text * attention_mask.unsqueeze(-1), dim=1)
        text_count = torch.sum(attention_mask, dim=1, keepdim=True).clamp(min=1)
        pooled_text = text_sum / text_count

        fused = torch.cat([pooled_audio, pooled_text], dim=1)

        # Task-Specific Processing
        joint_repr_arousal = self.fusion_layer_arousal(fused)
        joint_repr_dominance = self.fusion_layer_dominance(fused)

        shared_arousal = self.shared_fc_arousal(joint_repr_arousal)
        shared_dominance = self.shared_fc_dominance(joint_repr_dominance)

        output_arousal = self.output_branch_arousal(shared_arousal)
        output_dominance = self.output_branch_dominance(shared_dominance)

        # Scale outputs to [1, 5]
        scaled_arousal = 1.0 + 4.0 * torch.sigmoid(output_arousal)
        scaled_dominance = 1.0 + 4.0 * torch.sigmoid(output_dominance)

        return scaled_arousal, scaled_dominance

# Training function with backpropagation optimizations
def train_emotion_model(model, train_loader, val_loader, audio_model, num_epochs=15, lr=5e-5, max_norm=1.0):
    device = torch.device("cuda")
    model = model.to(device)
    audio_model = audio_model.to(device)

    # Differential learning rates
    optimizer = optim.AdamW([
        {"params": model.text_encoder.parameters(), "lr": 5e-6},
        {"params": [p for n, p in model.named_parameters() if "dominance" in n], "lr": 1e-4},  # Higher for dominance
        {"params": [p for n, p in model.named_parameters() if "text_encoder" not in n and "dominance" not in n], "lr": lr}
    ], weight_decay=1e-2)

    criterion = nn.HuberLoss()  # Robust to outliers
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=3, T_mult=2, eta_min=1e-6
    )
    warmup_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=len(train_loader) * num_epochs)

    best_val_loss = float('inf')
    patience_counter = 0
    max_patience = 10
    arousal_weight = 1.0
    dominance_weight = 1.0
    accumulation_steps = 2

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_arousal_loss = 0.0
        train_dominance_loss = 0.0
        num_train_batches = 0

        train_pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]")

        for i, (batch_inputs, labels) in enumerate(train_pbar):
            try:
                audio_values = batch_inputs['audio_values'].to(device)
                input_ids = batch_inputs['input_ids'].to(device)
                attention_mask = batch_inputs['attention_mask'].to(device)
                arousal_labels = labels['arousal'].to(device)
                dominance_labels = labels['dominance'].to(device)

                optimizer.zero_grad(set_to_none=True)

                with torch.no_grad():
                    hubert_out = audio_model(audio_values).last_hidden_state

                arousal_outputs, dominance_outputs = model(hubert_out, input_ids, attention_mask)

                arousal_loss = criterion(arousal_outputs.squeeze(), arousal_labels)
                dominance_loss = criterion(dominance_outputs.squeeze(), dominance_labels)

                total_loss = (arousal_weight * arousal_loss + dominance_weight * dominance_loss) / accumulation_steps

                total_loss.backward()

                if (i + 1) % accumulation_steps == 0:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=max_norm)
                    optimizer.step()
                    optimizer.zero_grad(set_to_none=True)

                # Monitor gradients every 100 batches
                if i % 100 == 0:
                    for name, param in model.named_parameters():
                        if param.grad is not None and ("arousal" in name or "dominance" in name):
                            print(f"{name}: Grad Norm = {param.grad.norm().item():.4f}")

                # Warmup for first 500 steps, then cosine annealing
                if i + epoch * len(train_loader) < 500:
                    warmup_scheduler.step()
                else:
                    scheduler.step()

                train_loss += total_loss.item() * accumulation_steps
                train_arousal_loss += arousal_loss.item()
                train_dominance_loss += dominance_loss.item()
                num_train_batches += 1

                train_pbar.set_postfix({
                    'loss': f"{total_loss.item() * accumulation_steps:.4f}",
                    'a_loss': f"{arousal_loss.item():.4f}",
                    'd_loss': f"{dominance_loss.item():.4f}"
                })

            except Exception as e:
                print(f"Error in training batch: {e}")
                continue

        if num_train_batches > 0:
            train_loss /= num_train_batches
            train_arousal_loss /= num_train_batches
            train_dominance_loss /= num_train_batches

            # Fixed and enhanced dynamic weighting
            total_task_loss = train_arousal_loss + train_dominance_loss
            if total_task_loss > 0:
                arousal_weight = 0.5 * (train_arousal_loss / total_task_loss)
                dominance_weight = 0.5 * (train_dominance_loss / total_task_loss)
                if train_dominance_loss > train_arousal_loss * 1.2:
                    dominance_weight *= 1.5
                    arousal_weight *= 0.5
                total_weight = arousal_weight + dominance_weight
                arousal_weight /= total_weight
                dominance_weight /= total_weight
                arousal_weight = max(0.25, min(0.75, arousal_weight))
                dominance_weight = max(0.25, min(0.75, dominance_weight))

        model.eval()
        val_loss = 0.0
        val_arousal_loss = 0.0
        val_dominance_loss = 0.0
        num_val_batches = 0

        val_pbar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]")

        with torch.no_grad():
            for batch_inputs, labels in val_pbar:
                try:
                    audio_values = batch_inputs['audio_values'].to(device)
                    input_ids = batch_inputs['input_ids'].to(device)
                    attention_mask = batch_inputs['attention_mask'].to(device)
                    arousal_labels = labels['arousal'].to(device)
                    dominance_labels = labels['dominance'].to(device)

                    hubert_out = audio_model(audio_values).last_hidden_state

                    arousal_outputs, dominance_outputs = model(hubert_out, input_ids, attention_mask)

                    arousal_loss = criterion(arousal_outputs.squeeze(), arousal_labels)
                    dominance_loss = criterion(dominance_outputs.squeeze(), dominance_labels)
                    total_loss = 0.5 * arousal_loss + 0.5 * dominance_loss

                    val_loss += total_loss.item()
                    val_arousal_loss += arousal_loss.item()
                    val_dominance_loss += dominance_loss.item()
                    num_val_batches += 1

                    val_pbar.set_postfix({
                        'loss': f"{total_loss.item():.4f}",
                        'a_loss': f"{arousal_loss.item():.4f}",
                        'd_loss': f"{dominance_loss.item():.4f}"
                    })

                except Exception as e:
                    print(f"Error in validation batch: {e}")
                    continue

        if num_val_batches > 0:
            val_loss /= num_val_batches
            val_arousal_loss /= num_val_batches
            val_dominance_loss /= num_val_batches

        print(f"Epoch {epoch+1}/{num_epochs} Results:")
        print(f"  Train Loss: {train_loss:.4f} (Arousal: {train_arousal_loss:.4f}, Dominance: {train_dominance_loss:.4f})")
        print(f"  Val Loss: {val_loss:.4f} (Arousal: {val_arousal_loss:.4f}, Dominance: {val_dominance_loss:.4f})")
        print(f"  Loss Weights - Arousal: {arousal_weight:.4f}, Dominance: {dominance_weight:.4f}")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), '/content/best_arousal_dominance_model.pth')
            print(f"  Saved best model (val_loss: {val_loss:.4f})")
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= max_patience:
                print(f"Early stopping after {epoch+1} epochs without improvement")
                break

    return best_val_loss

# Evaluation function (completed)
def evaluate_emotion_model(model, test_loader, audio_model, device):
    model.eval()
    criterion = nn.HuberLoss()
    test_loss = 0.0
    test_arousal_loss = 0.0
    test_dominance_loss = 0.0
    num_batches = 0

    all_arousal_preds = []
    all_arousal_labels = []
    all_dominance_preds = []
    all_dominance_labels = []

    with torch.no_grad():
        for batch_inputs, labels in test_loader:
            try:
                audio_values = batch_inputs['audio_values'].to(device)
                input_ids = batch_inputs['input_ids'].to(device)
                attention_mask = batch_inputs['attention_mask'].to(device)
                arousal_labels = labels['arousal'].to(device)
                dominance_labels = labels['dominance'].to(device)

                hubert_out = audio_model(audio_values).last_hidden_state

                arousal_outputs, dominance_outputs = model(hubert_out, input_ids, attention_mask)

                arousal_loss = criterion(arousal_outputs.squeeze(), arousal_labels)
                dominance_loss = criterion(dominance_outputs.squeeze(), dominance_labels)

                total_loss = 0.5 * arousal_loss + 0.5 * dominance_loss

                test_loss += total_loss.item()
                test_arousal_loss += arousal_loss.item()
                test_dominance_loss += dominance_loss.item()
                num_batches += 1

                all_arousal_preds.append(arousal_outputs.squeeze().cpu().numpy())
                all_arousal_labels.append(arousal_labels.cpu().numpy())
                all_dominance_preds.append(dominance_outputs.squeeze().cpu().numpy())
                all_dominance_labels.append(dominance_labels.cpu().numpy())

            except Exception as e:
                print(f"Error in test batch: {e}")
                continue

    if num_batches > 0:
        test_loss /= num_batches
        test_arousal_loss /= num_batches
        test_dominance_loss /= num_batches

        from scipy.stats import pearsonr
        all_arousal_preds = np.concatenate(all_arousal_preds)
        all_arousal_labels = np.concatenate(all_arousal_labels)
        all_dominance_preds = np.concatenate(all_dominance_preds)
        all_dominance_labels = np.concatenate(all_dominance_labels)

        arousal_pcc, _ = pearsonr(all_arousal_preds, all_arousal_labels)
        dominance_pcc, _ = pearsonr(all_dominance_preds, all_dominance_labels)
    else:
        test_loss = float('inf')
        test_arousal_loss = float('inf')
        test_dominance_loss = float('inf')
        arousal_pcc = 0.0
        dominance_pcc = 0.0

    return test_loss, test_arousal_loss, test_dominance_loss, arousal_pcc, dominance_pcc

# Main execution
def main():
    device = torch.device("cuda")

    # Configuration
    csv_file = "/content/sample_data/updated_iemocap_metadata_colab.csv"
    audio_feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/hubert-base-ls960")
    text_tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

    hubert_model = HubertModel.from_pretrained("facebook/hubert-base-ls960")
    for param in hubert_model.parameters():
        param.requires_grad = False
    hubert_model.eval()
    hubert_model = hubert_model.to(device)

    # Dataset and loaders
    train_dataset = MultimodalIEMOCAPDataset(
        csv_file=csv_file,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=True,
        max_audio_samples=128000
    )
    val_dataset = MultimodalIEMOCAPDataset(
        csv_file=csv_file,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=False,
        max_audio_samples=128000
    )
    test_dataset = MultimodalIEMOCAPDataset(
        csv_file=csv_file,
        audio_feature_extractor=audio_feature_extractor,
        text_tokenizer=text_tokenizer,
        augment=False,
        max_audio_samples=128000
    )

    train_size = int(0.8 * len(train_dataset))
    val_size = int(0.1 * len(train_dataset))
    test_size = len(train_dataset) - train_size - val_size
    train_dataset, _, _ = random_split(
        train_dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    _, val_dataset, _ = random_split(
        val_dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    _, _, test_dataset = random_split(
        test_dataset, [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )

    train_loader = DataLoader(
        train_dataset,
        batch_size=8,
        shuffle=True,
        num_workers=0,
        pin_memory=True,
        collate_fn=dynamic_collate_fn
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=dynamic_collate_fn
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        collate_fn=dynamic_collate_fn
    )

    # Initialize the model
    model = MultimodalArousalDominanceModel(
        audio_dim=768,
        text_dim=768,
        hidden_dim=192,
        num_heads=6,
        num_layers=2,
        dropout=0.5
    ).to(device)

    # Training
    print("Training Arousal and Dominance model...")
    best_val_loss = train_emotion_model(
        model,
        train_loader,
        val_loader,
        hubert_model,
        num_epochs=15,
        lr=5e-5,
        max_norm=1.0
    )

    # Evaluation
    model.load_state_dict(torch.load('/content/best_arousal_dominance_model.pth'))
    test_loss, test_arousal_loss, test_dominance_loss, arousal_pcc, dominance_pcc = evaluate_emotion_model(
        model, test_loader, hubert_model, device
    )

    print("\nTest Results:")
    print(f"  Overall Test Loss: {test_loss:.4f}")
    print(f"  Arousal MSE: {test_arousal_loss:.4f}, PCC: {arousal_pcc:.4f}")
    print(f"  Dominance MSE: {test_dominance_loss:.4f}, PCC: {dominance_pcc:.4f}")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Training Arousal and Dominance model...


Epoch 1/15 [Train]:   0%|          | 1/1004 [00:00<10:42,  1.56it/s, loss=0.7697, a_loss=0.3017, d_loss=0.4680]

fusion_layer_arousal.0.weight: Grad Norm = 1.7369
fusion_layer_arousal.0.bias: Grad Norm = 0.7337
fusion_layer_arousal.1.weight: Grad Norm = 0.0602
fusion_layer_arousal.1.bias: Grad Norm = 0.0610
fusion_layer_arousal.4.weight: Grad Norm = 1.8725
fusion_layer_arousal.4.bias: Grad Norm = 0.1073
fusion_layer_dominance.0.weight: Grad Norm = 1.5990
fusion_layer_dominance.0.bias: Grad Norm = 0.6528
fusion_layer_dominance.1.weight: Grad Norm = 0.0603
fusion_layer_dominance.1.bias: Grad Norm = 0.0520
fusion_layer_dominance.4.weight: Grad Norm = 1.7725
fusion_layer_dominance.4.bias: Grad Norm = 0.1039
shared_fc_arousal.0.weight: Grad Norm = 1.3586
shared_fc_arousal.0.bias: Grad Norm = 0.1959
shared_fc_arousal.1.weight: Grad Norm = 0.0633
shared_fc_arousal.1.bias: Grad Norm = 0.0611
shared_fc_dominance.0.weight: Grad Norm = 1.3562
shared_fc_dominance.0.bias: Grad Norm = 0.1830
shared_fc_dominance.1.weight: Grad Norm = 0.0766
shared_fc_dominance.1.bias: Grad Norm = 0.0637
output_branch_arousal.0.

Epoch 1/15 [Train]:  10%|█         | 101/1004 [00:56<08:01,  1.88it/s, loss=0.5750, a_loss=0.1902, d_loss=0.3848]

...

fusion_layer_arousal.0.weight: Grad Norm = 0.0520
fusion_layer_arousal.0.bias: Grad Norm = 0.0127
fusion_layer_arousal.1.weight: Grad Norm = 0.0022
fusion_layer_arousal.1.bias: Grad Norm = 0.0021
fusion_layer_arousal.4.weight: Grad Norm = 0.0567
fusion_layer_arousal.4.bias: Grad Norm = 0.0033
fusion_layer_dominance.0.weight: Grad Norm = 0.1382
fusion_layer_dominance.0.bias: Grad Norm = 0.0263
fusion_layer_dominance.1.weight: Grad Norm = 0.0081
fusion_layer_dominance.1.bias: Grad Norm = 0.0077
fusion_layer_dominance.4.weight: Grad Norm = 0.1366
fusion_layer_dominance.4.bias: Grad Norm = 0.0090
shared_fc_arousal.0.weight: Grad Norm = 0.0494
shared_fc_arousal.0.bias: Grad Norm = 0.0050
shared_fc_arousal.1.weight: Grad Norm = 0.0030
shared_fc_arousal.1.bias: Grad Norm = 0.0032
shared_fc_dominance.0.weight: Grad Norm = 0.1267
shared_fc_dominance.0.bias: Grad Norm = 0.0105
shared_fc_dominance.1.weight: Grad Norm = 0.0104
shared_fc_dominance.1.bias: Grad Norm = 0.0105
output_branch_arousal.0.

Epoch 15/15 [Train]: 100%|█████████▉| 1001/1004 [09:15<00:01,  1.98it/s, loss=0.1470, a_loss=0.0749, d_loss=0.1710]

fusion_layer_arousal.0.weight: Grad Norm = 0.0662
fusion_layer_arousal.0.bias: Grad Norm = 0.0143
fusion_layer_arousal.1.weight: Grad Norm = 0.0030
fusion_layer_arousal.1.bias: Grad Norm = 0.0026
fusion_layer_arousal.4.weight: Grad Norm = 0.0608
fusion_layer_arousal.4.bias: Grad Norm = 0.0036
fusion_layer_dominance.0.weight: Grad Norm = 0.1021
fusion_layer_dominance.0.bias: Grad Norm = 0.0148
fusion_layer_dominance.1.weight: Grad Norm = 0.0057
fusion_layer_dominance.1.bias: Grad Norm = 0.0052
fusion_layer_dominance.4.weight: Grad Norm = 0.1136
fusion_layer_dominance.4.bias: Grad Norm = 0.0056
shared_fc_arousal.0.weight: Grad Norm = 0.0527
shared_fc_arousal.0.bias: Grad Norm = 0.0053
shared_fc_arousal.1.weight: Grad Norm = 0.0032
shared_fc_arousal.1.bias: Grad Norm = 0.0033
shared_fc_dominance.0.weight: Grad Norm = 0.1190
shared_fc_dominance.0.bias: Grad Norm = 0.0087
shared_fc_dominance.1.weight: Grad Norm = 0.0111
shared_fc_dominance.1.bias: Grad Norm = 0.0093
output_branch_arousal.0.

Epoch 15/15 [Val]: 100%|██████████| 126/126 [00:45<00:00,  2.79it/s, loss=0.2454, a_loss=0.1459, d_loss=0.3449]


Epoch 15/15 Results:
  Train Loss: 0.1570 (Arousal: 0.1185, Dominance: 0.1698)
  Val Loss: 0.1363 (Arousal: 0.0996, Dominance: 0.1730)
  Loss Weights - Arousal: 0.2500, Dominance: 0.7500
  Saved best model (val_loss: 0.1363)

Test Results:
  Overall Test Loss: 0.1360
  Arousal MSE: 0.1046, PCC: 0.7292
  Dominance MSE: 0.1675, PCC: 0.6229


In [ ]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer, HubertModel, Wav2Vec2FeatureExtractor
import torchaudio
import os
import shutil
from pathlib import Path

# ValenceRegressor
class ValenceRegressor(nn.Module):
    def __init__(self, audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5):
        super().__init__()
        self.audio_transformer = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=audio_dim, nhead=num_heads, dim_feedforward=hidden_dim*4, dropout=dropout, batch_first=True)
            for _ in range(num_layers)
        ])
        self.audio_layer_norm = nn.LayerNorm(audio_dim)
        self.audio_attention_pool = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim*2), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_dim*2, 1)
        )
        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        for param in list(self.text_encoder.parameters())[-2:]:
            param.requires_grad = True
        self.audio_projection = nn.Linear(audio_dim, hidden_dim)
        self.text_projection = nn.Linear(text_dim, hidden_dim)
        self.audio_to_text_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads//2, dropout=dropout, batch_first=True)
        self.text_to_audio_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads//2, dropout=dropout, batch_first=True)
        self.audio_gate = nn.Sequential(nn.Linear(hidden_dim*2, hidden_dim), nn.Sigmoid())
        self.text_gate = nn.Sequential(nn.Linear(hidden_dim*2, hidden_dim), nn.Sigmoid())
        self.fusion_layer = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2), nn.LayerNorm(hidden_dim*2), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_dim*2, hidden_dim)
        )
        self.shared_fc = nn.Sequential(nn.Linear(hidden_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.GELU(), nn.Dropout(dropout))
        self.output_branch = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2), nn.LayerNorm(hidden_dim//2), nn.GELU(), nn.Dropout(dropout*0.5), nn.Linear(hidden_dim//2, 1)
        )

    def audio_attention_pooling(self, x, audio_mask=None):
        weights = self.audio_attention_pool(x)
        if audio_mask is not None:
            weights = weights.masked_fill(~audio_mask.bool().unsqueeze(-1), float('-inf'))
        weights = torch.softmax(weights, dim=1)
        output = torch.bmm(weights.transpose(1, 2), x)
        return output.squeeze(1)

    def forward(self, audio_features, input_ids, attention_mask):
        audio_mask = (audio_features.abs().sum(dim=-1) > 1e-6)
        audio_repr = audio_features
        for layer in self.audio_transformer:
            audio_key_padding_mask = (~audio_mask).float()
            audio_repr = layer(audio_repr, src_key_padding_mask=audio_key_padding_mask)
        audio_repr = self.audio_layer_norm(audio_repr)
        text_outputs = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        text_repr = text_outputs.last_hidden_state
        audio_proj = self.audio_projection(audio_repr)
        text_proj = self.text_projection(text_repr)
        audio_attended_text, _ = self.audio_to_text_attention(
            query=audio_proj, key=text_proj, value=text_proj, key_padding_mask=(1 - attention_mask).bool()
        )
        text_attended_audio, _ = self.text_to_audio_attention(
            query=text_proj, key=audio_proj, value=audio_proj, key_padding_mask=(~audio_mask).bool()
        )
        audio_concat = torch.cat([audio_proj, audio_attended_text], dim=-1)
        text_concat = torch.cat([text_proj, text_attended_audio], dim=-1)
        audio_gate_value = self.audio_gate(audio_concat)
        text_gate_value = self.text_gate(text_concat)
        gated_audio = audio_proj * audio_gate_value
        gated_text = text_proj * text_gate_value
        pooled_audio = self.audio_attention_pooling(gated_audio, audio_mask)
        text_sum = torch.sum(gated_text * attention_mask.unsqueeze(-1), dim=1)
        text_count = torch.sum(attention_mask, dim=1, keepdim=True).clamp(min=1)
        pooled_text = text_sum / text_count
        fused = torch.cat([pooled_audio, pooled_text], dim=1)
        joint_repr = self.fusion_layer(fused)
        shared = self.shared_fc(joint_repr)
        output = self.output_branch(shared)
        scaled_output = 1.0 + 4.0 * torch.sigmoid(output)
        return scaled_output

# MultimodalArousalDominanceModel
class MultimodalArousalDominanceModel(nn.Module):
    def __init__(self, audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5):
        super().__init__()
        self.audio_transformer = nn.ModuleList([
            nn.TransformerEncoderLayer(d_model=audio_dim, nhead=num_heads, dim_feedforward=hidden_dim*4, dropout=dropout, batch_first=True)
            for _ in range(num_layers)
        ])
        self.audio_layer_norm = nn.LayerNorm(audio_dim)
        self.audio_attention_pool = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim*2), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_dim*2, 1)
        )
        self.text_encoder = BertModel.from_pretrained("bert-base-uncased")
        for param in self.text_encoder.parameters():
            param.requires_grad = False
        for param in list(self.text_encoder.parameters())[-2:]:
            param.requires_grad = True
        self.audio_projection = nn.Linear(audio_dim, hidden_dim)
        self.text_projection = nn.Linear(text_dim, hidden_dim)
        self.audio_to_text_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads//2, dropout=dropout, batch_first=True)
        self.text_to_audio_attention = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=num_heads//2, dropout=dropout, batch_first=True)
        self.audio_gate = nn.Sequential(nn.Linear(hidden_dim*2, hidden_dim), nn.Sigmoid())
        self.text_gate = nn.Sequential(nn.Linear(hidden_dim*2, hidden_dim), nn.Sigmoid())
        self.fusion_layer_arousal = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2), nn.LayerNorm(hidden_dim*2), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_dim*2, hidden_dim)
        )
        self.fusion_layer_dominance = nn.Sequential(
            nn.Linear(hidden_dim*2, hidden_dim*2), nn.LayerNorm(hidden_dim*2), nn.GELU(), nn.Dropout(dropout), nn.Linear(hidden_dim*2, hidden_dim)
        )
        self.shared_fc_arousal = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.GELU(), nn.Dropout(dropout)
        )
        self.shared_fc_dominance = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim), nn.LayerNorm(hidden_dim), nn.GELU(), nn.Dropout(dropout)
        )
        self.output_branch_arousal = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2), nn.LayerNorm(hidden_dim//2), nn.GELU(), nn.Dropout(dropout*0.5), nn.Linear(hidden_dim//2, 1)
        )
        self.output_branch_dominance = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim//2), nn.LayerNorm(hidden_dim//2), nn.GELU(), nn.Dropout(dropout*0.5), nn.Linear(hidden_dim//2, 1)
        )

    def audio_attention_pooling(self, x, audio_mask=None):
        weights = self.audio_attention_pool(x)
        if audio_mask is not None:
            weights = weights.masked_fill(~audio_mask.bool().unsqueeze(-1), float('-inf'))
        weights = torch.softmax(weights, dim=1)
        output = torch.bmm(weights.transpose(1, 2), x)
        return output.squeeze(1)

    def forward(self, audio_features, input_ids, attention_mask):
        audio_mask = (audio_features.abs().sum(dim=-1) > 1e-6)
        audio_repr = audio_features
        for layer in self.audio_transformer:
            audio_key_padding_mask = (~audio_mask).float()
            audio_repr = layer(audio_repr, src_key_padding_mask=audio_key_padding_mask)
        audio_repr = self.audio_layer_norm(audio_repr)
        text_outputs = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        text_repr = text_outputs.last_hidden_state
        audio_proj = self.audio_projection(audio_repr)
        text_proj = self.text_projection(text_repr)
        audio_attended_text, _ = self.audio_to_text_attention(
            query=audio_proj, key=text_proj, value=text_proj, key_padding_mask=(1 - attention_mask).bool()
        )
        text_attended_audio, _ = self.text_to_audio_attention(
            query=text_proj, key=audio_proj, value=audio_proj, key_padding_mask=(~audio_mask).bool()
        )
        audio_concat = torch.cat([audio_proj, audio_attended_text], dim=-1)
        text_concat = torch.cat([text_proj, text_attended_audio], dim=-1)
        audio_gate_value = self.audio_gate(audio_concat)
        text_gate_value = self.text_gate(text_concat)
        gated_audio = audio_proj * audio_gate_value
        gated_text = text_proj * text_gate_value
        pooled_audio = self.audio_attention_pooling(gated_audio, audio_mask)
        text_sum = torch.sum(gated_text * attention_mask.unsqueeze(-1), dim=1)
        text_count = torch.sum(attention_mask, dim=1, keepdim=True).clamp(min=1)
        pooled_text = text_sum / text_count
        fused = torch.cat([pooled_audio, pooled_text], dim=1)
        joint_repr_arousal = self.fusion_layer_arousal(fused)
        joint_repr_dominance = self.fusion_layer_dominance(fused)
        shared_arousal = self.shared_fc_arousal(joint_repr_arousal)
        shared_dominance = self.shared_fc_dominance(joint_repr_dominance)
        output_arousal = self.output_branch_arousal(shared_arousal)
        output_dominance = self.output_branch_dominance(shared_dominance)
        scaled_arousal = 1.0 + 4.0 * torch.sigmoid(output_arousal)
        scaled_dominance = 1.0 + 4.0 * torch.sigmoid(output_dominance)
        return scaled_arousal, scaled_dominance

# Function to clear Hugging Face cache
def clear_huggingface_cache():
    cache_dir = Path.home() / ".cache" / "huggingface" / "transformers"
    if cache_dir.exists():
        shutil.rmtree(cache_dir)
        print(f"Cleared cache at {cache_dir}")

# Function to load feature extractor with retry
def load_feature_extractor(model_name, max_retries=3):
    for attempt in range(max_retries):
        try:
            return Wav2Vec2FeatureExtractor.from_pretrained(model_name)
        except (OSError, ValueError) as e:
            print(f"Attempt {attempt+1}/{max_retries} failed: {e}")
            if attempt < max_retries - 1:
                clear_huggingface_cache()
    raise RuntimeError(f"Failed to load Wav2Vec2FeatureExtractor after {max_retries} attempts")

# Function to extract audio features using HuBERT
def extract_hubert_features(audio_path, processor, hubert_model, device, sampling_rate=16000, max_audio_samples=128000):
    audio, sr = torchaudio.load(audio_path)
    if sr != sampling_rate:
        audio = torchaudio.transforms.Resample(sr, sampling_rate)(audio)
    audio = audio.squeeze(0)
    if audio.dim() > 1:
        audio = audio[0]
    if audio.size(0) > max_audio_samples:
        audio = audio[:max_audio_samples]
    elif audio.size(0) < max_audio_samples:
        audio = torch.nn.functional.pad(audio, (0, max_audio_samples - audio.size(0)))
    audio = audio.cpu().numpy()
    inputs = processor(audio, sampling_rate=sampling_rate, return_tensors="pt", padding=False, truncation=False)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = hubert_model(**inputs)
    return outputs.last_hidden_state

# Function to preprocess inputs
def preprocess_inputs(text, audio_path, tokenizer, processor, hubert_model, device, max_length=512):
    encoding = tokenizer(text, max_length=max_length, padding='max_length', truncation=True, return_tensors='pt')
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    audio_features = extract_hubert_features(audio_path, processor, hubert_model, device)
    return input_ids, attention_mask, audio_features

# Function to load models
def load_models(valence_checkpoint_path, ad_checkpoint_path, device):
    valence_model = ValenceRegressor(audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5)
    valence_checkpoint = torch.load(valence_checkpoint_path, map_location=device)
    valence_model.load_state_dict(valence_checkpoint['model_state_dict'])
    if torch.cuda.device_count() > 1:
        valence_model = nn.DataParallel(valence_model)
    valence_model.to(device)
    valence_model.eval()

    ad_model = MultimodalArousalDominanceModel(audio_dim=768, text_dim=768, hidden_dim=192, num_heads=6, num_layers=2, dropout=0.5)
    ad_checkpoint = torch.load(ad_checkpoint_path, map_location=device)
    ad_model.load_state_dict(ad_checkpoint['model_state_dict'])
    if torch.cuda.device_count() > 1:
        ad_model = nn.DataParallel(ad_model)
    ad_model.to(device)
    ad_model.eval()

    return valence_model, ad_model

# Main function for prediction
def predict_emotions(audio_path, transcription, valence_checkpoint_path, ad_checkpoint_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    processor = load_feature_extractor('facebook/hubert-base-ls960')
    hubert_model = HubertModel.from_pretrained('facebook/hubert-base-ls960').to(device)
    hubert_model.eval()

    valence_model, ad_model = load_models(valence_checkpoint_path, ad_checkpoint_path, device)

    input_ids, attention_mask, audio_features = preprocess_inputs(
        text=transcription,
        audio_path=audio_path,
        tokenizer=tokenizer,
        processor=processor,
        hubert_model=hubert_model,
        device=device,
        max_length=512
    )

    with torch.no_grad():
        valence_pred = valence_model(audio_features, input_ids, attention_mask)
        arousal_pred, dominance_pred = ad_model(audio_features, input_ids, attention_mask)

    return {
        'valence': valence_pred.item(),
        'arousal': arousal_pred.item(),
        'dominance': dominance_pred.item()
    }

# Example usage
if __name__ == "__main__":
    valence_checkpoint_path = 'path/to/valence_model_checkpoint.pth'  # Replace
    ad_checkpoint_path = '/content/best_arousal_dominance_model.pth'
    audio_path = 'path/to/audio.wav'  # Replace
    transcription = "This is a sample utterance from IEMOCAP."

    predictions = predict_emotions(audio_path, transcription, valence_checkpoint_path, ad_checkpoint_path)

    print(f"Predicted Valence: {predictions['valence']:.4f}")
    print(f"Predicted Arousal: {predictions['arousal']:.4f}")
    print(f"Predicted Dominance: {predictions['dominance']:.4f}")